# 01: Register Unity Catalog Functions (DEPRECATED)

⚠️ **THIS NOTEBOOK IS DEPRECATED** ⚠️

This notebook was used to register custom UC Functions for Genie access, but we've migrated to the official **GenieAgent** pattern from `databricks_langchain`.

## Why Deprecated?

The custom UC Function approach had authentication issues because:
- UC Functions run in serverless Spark (no WorkspaceClient auth support)
- Serverless Spark with on-behalf-of (OBO) is not yet supported

## New Approach

We now use **GenieAgent** from `databricks_langchain` which:
- ✅ Handles authentication automatically
- ✅ Official Databricks pattern
- ✅ No custom UC Function registration needed
- ✅ Simpler implementation

## What to Use Instead

**Skip this notebook** and go directly to:
- `02-create-agent.ipynb` - Creates multi-agent system with GenieAgent
- `03-test-agent.ipynb` - Tests the agent

---

**Note**: This notebook is kept for reference only. The UC Function implementations have been removed.

## Install Dependencies

In [ ]:
%pip install --quiet --upgrade unitycatalog-ai databricks-sdk
dbutils.library.restartPython()

## Setup Paths and Imports

In [ ]:
import sys
sys.path.append('../src')

from tools.customer_behavior import register_customer_behavior_tool
from tools.inventory import register_inventory_tool
from config.agent_config import AGENT_CONFIG

print("✅ Imports successful")

## Register UC Functions

Register both Genie tools as Unity Catalog Functions.

In [ ]:
# Get catalog and schema from config
catalog = AGENT_CONFIG["tools_catalog"]
schema = AGENT_CONFIG["tools_schema"]

print(f"Registering UC Functions in {catalog}.{schema}...\n")

# Register Customer Behavior tool
customer_func = register_customer_behavior_tool(catalog=catalog, schema=schema)

# Register Inventory tool
inventory_func = register_inventory_tool(catalog=catalog, schema=schema)

print(f"\n✅ Registration complete!")
print(f"\nRegistered functions:")
print(f"  - {customer_func}")
print(f"  - {inventory_func}")

## Verify Registration

List all functions in the schema to verify.

In [ ]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

# List functions in schema
functions = w.functions.list(catalog_name=catalog, schema_name=schema)

print(f"\nFunctions in {catalog}.{schema}:")
for func in functions:
    print(f"  - {func.full_name}")

## Next Steps

✅ UC Functions registered and validated!

**If validation tests passed**:
- Open `02-create-agent.ipynb` to create the agent using these UC Functions

**If validation tests failed**:
- Check UC Function code in `../src/tools/customer_behavior.py` and `../src/tools/inventory.py`
- Verify authentication logic in WorkspaceClient() initialization
- Re-run this notebook after fixing

In [ ]:
print("="*60)
print("UC FUNCTION VALIDATION SUMMARY")
print("="*60)
print(f"✅ Registered: {customer_func}")
print(f"✅ Registered: {inventory_func}")
print("\nIf both tests passed above:")
print("  ✅ Authentication is working correctly")
print("  ✅ UC Functions can access Genie API")
print("  ✅ Ready to create agent in 02-create-agent.ipynb")
print("\nIf tests failed with authentication errors:")
print("  ❌ Re-check UC Function code in ../src/tools/")
print("  ❌ Ensure WorkspaceClient() credentials strategy is correct")
print("="*60)

## Test Summary

In [ ]:
print("Testing Inventory UC Function...\n")

# Test query
test_query = "What products are currently at risk of overstock?"

# Call UC Function via SQL
result = spark.sql(f"""
    SELECT {inventory_func}('{test_query}') as response
""").collect()[0]['response']

print(f"Query: {test_query}")
print(f"\nResponse:\n{result}")

# Check for errors
if "[Error]" in result and "cannot configure default credentials" in result:
    print("\n❌ AUTHENTICATION ERROR - UC Function cannot access Genie API")
    print("   This means the authentication fix didn't work. Check UC Function code.")
elif "[Error]" in result:
    print(f"\n⚠️  UC Function returned an error (but not auth-related)")
else:
    print("\n✅ Inventory UC Function works!")

In [ ]:
print("Testing Customer Behavior UC Function...\n")

# Test query
test_query = "What are the top 5 products by cart abandonment rate?"

# Call UC Function via SQL
result = spark.sql(f"""
    SELECT {customer_func}('{test_query}') as response
""").collect()[0]['response']

print(f"Query: {test_query}")
print(f"\nResponse:\n{result}")

# Check for errors
if "[Error]" in result and "cannot configure default credentials" in result:
    print("\n❌ AUTHENTICATION ERROR - UC Function cannot access Genie API")
    print("   This means the authentication fix didn't work. Check UC Function code.")
elif "[Error]" in result:
    print(f"\n⚠️  UC Function returned an error (but not auth-related)")
else:
    print("\n✅ Customer Behavior UC Function works!")

## Test UC Functions

Validate that registered UC Functions work correctly with authentication.